# Transcrire un fichier de sous-titres VTT simple en TextGrid

## import des modules

In [1]:
import pandas as pd
# import numpy as np
import codecs,re

# Lecture d'un fichier VTT
- ouvrir en UTF8
- supprimer les lignes vides
- parcourir toutes les lignes restantes
    - mettre un numéro quand on trouve une ligne du type tc --> tc
        - récupérer le tc début avant -->
        - récupérer le tc fin après -->
    - ajouter le texte quand on a une ligne sans -->

In [2]:
def lireVTT(nFile,lHeader=""):
    result={}
    with codecs.open(nFile,"r",encoding="utf8") as inFile:
        subSRT=[l.strip() for l in inFile.readlines() if l.strip()!=""]
    
    print subSRT[:5]
    n=0
    for l in subSRT:
        # print l
        m=re.match(ur"(.*) --> .*",l)
        if m:
            n+=1
            result[n]=[]
            tDep=l.split(" --> ")[0]
            result[n].append(tDep)
            tFin=l.split(" --> ")[1]
            result[n].append(tFin)
        else:
            m=re.match(lHeader+ur"(.*)$",l)
            if m:
                result[n].append(m.group(1))
    return result

# Faire une DataFrame avec le VTT
- récupérer le dictionnaire de lireVTT
- transformer le dictionnaire en DataFrame

In [3]:
fWhisperSRT=ur"/Users/gilles/Downloads/Joséphine/15_20.vtt"
prefWhisperSRT=ur""
dSRT=lireVTT(fWhisperSRT,prefWhisperSRT)
dfWhisper=pd.DataFrame.from_dict(dSRT,orient="index",columns=["start","end","text"])
# dfWhisper

[u'00:15:03.280 --> 00:15:06.240', u'entre le num\xe9ro 8 et le num\xe9ro 9 fran\xe7ais.', u'00:15:06.240 --> 00:15:08.520', u"Et c'est pour \xe7a qu'il s'\xe9carte et qu'il s'allonge,", u'00:15:08.520 --> 00:15:11.240']


# Préparer les timecodes
- un tc 0 pour calculer des différences
- un startTime et un endTime au format datetime

In [4]:
tcZero=pd.to_datetime("00:00:00.000",format='%H:%M:%S.%f')
dfWhisper["startTime"]=pd.to_datetime(dfWhisper["start"],format='%H:%M:%S.%f')
dfWhisper["endTime"]=pd.to_datetime(dfWhisper["end"],format='%H:%M:%S.%f')

## transformer les start et end en secondes
- start et end sont recalculés à partir de startTime et endTime
- la différence entre deux datetime peut être exprimée en secondes

In [5]:
dfWhisper["start"]=(dfWhisper.startTime-tcZero).dt.total_seconds()
dfWhisper["end"]=(dfWhisper.endTime-tcZero).dt.total_seconds()
# dfWhisper

# Calculer les entêtes
- xmin est le min des start
- xmax est le max des end
- intervals est le nombre de tours

In [6]:
xmin=dfWhisper.start.min()
xmax=dfWhisper.end.max()
intervals=dfWhisper.index.max()

In [7]:
entete='''File type = "ooTextFile"
Object class = "TextGrid"

xmin = %f 
xmax = %f 
tiers? <exists> 
size = 1

item []: 
    item [1]:
        class = "IntervalTier" 
        name = "Trans" 
        xmin = %f 
        xmax = %f 
        intervals: size = %d '''%(xmin,xmax,xmin,xmax,intervals)

# Faire le tour
Il y a 4 composants dans un tour :
1. intervals : un numéro séquentiel
1. xmin : start
1. xmax : end
1. text : le texte du tour

In [8]:
def makeTour(x):
    print "\tintervals [%d]"%(x.name)
    print "\txmin = %f"%x.start
    print "\txmax = %f"%x.end
    print '\ttext = "%s"'%x.text
    return

# Imprimer le TextGrid

In [9]:
print entete
temp=dfWhisper.apply(makeTour,axis=1)

File type = "ooTextFile"
Object class = "TextGrid"

xmin = 903.280000 
xmax = 1205.660000 
tiers? <exists> 
size = 1

item []: 
    item [1]:
        class = "IntervalTier" 
        name = "Trans" 
        xmin = 903.280000 
        xmax = 1205.660000 
        intervals: size = 78 
	intervals [1]
	xmin = 903.280000
	xmax = 906.240000
	text = "entre le numéro 8 et le numéro 9 français."
	intervals [2]
	xmin = 906.240000
	xmax = 908.520000
	text = "Et c'est pour ça qu'il s'écarte et qu'il s'allonge,"
	intervals [3]
	xmin = 908.520000
	xmax = 911.240000
	text = "pour obliger le neuf argentaire à faire un gramme d'équipe."
	intervals [4]
	xmin = 911.240000
	xmax = 913.040000
	text = "Oh ben la poussée argentine est forte hein !"
	intervals [5]
	xmin = 913.640000
	xmax = 918.120000
	text = "Avec Pichotte, astucieux malin pour lancer Hernandez."
	intervals [6]
	xmin = 918.120000
	xmax = 923.080000
	text = "Quel coquin ce Pichotte, le petit général."
	intervals [7]
	xmin = 923.080000
	xmax = 